In [1]:
import numpy as np
import random as rd
import time

In [16]:
class Environment():
    
    #fields:
    #a grid of size defined by parameter
    #b, a tuple repsenting the delivery location
    #a, a tuple representing the package location
    #size, an int representing the horizontal and vertical dimensions of the grid
    def __init__(self, size:int, a = None):
        self.size = size
        self.b = (size-1, size-1)
        self.grid, self.a = self.setup_grid(size, a)
        #potentially need a rewards directory for grid states:
        #give, say, 50 for (a, agent.collected = 0), and 50 for (a, agent.collected = 1), and -1 for ANY other states
        

    #method to set up the original grid including a location
    def setup_grid(self, size, a):
        grid = np.zeros((size,size))
        if a is None:
            x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            while (x,y) == self.b:
                x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            grid[x,y] = 1  #A represented by 1
        else:
            x, y = a
            grid[x,y] = 1
        grid[size-1,size-1] = 2  #B represented by 2
        return grid, (x,y)
    
    #method to add start location of agent to grid as well
 #TODO: consider whether to initialise agent as part of environment initialisation
 #pass agent coordinates into constructor instead lmao DENSE
    def set_agent_start(self, agent):
        self.agent_coords = (agent.x, agent.y)
    
    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, agent_move):
        x, y = self.agent_coords
        self.grid[x,y] = 0
        self.agent_coords = agent_move
        self.grid[agent_move] = -1   

#An agent class. Currently barebones
class Agent():
    def __init__(self, environ):
        self.environ = environ
        self.x, self.y = rd.randint(0, environ.size-1), rd.randint(0, environ.size-1)
        while ((self.x, self.y) == (environ.a or environ.b)):
            self.x, self.y = rd.randint(0, environ.size-1), rd.randint(0, environ.size-1)
        self.opts = ["up", "down", "left", "right"]
        self.package = 0

    def choose_best_option():
        #some sort of policy calculation
        #return: best option
        pass

    #Ensure that moves
    def move(self, direction):
        if direction=="left":
            if self.x-1 >= 0:
                self.x = self.x-1
        if direction=="right":
            if self.x+1 < self.environ.size:
                self.x = self.x+1
        if direction=="up":
            if self.y-1 >= 0:
                self.y = self.y-1
        if direction=="down":
            if self.y+1 < self.environ.size:
                self.y=self.y+1
        return self.x, self.y
    
#Class to store hyperparameters
class Hypers():
    def __init__(self, eps, gamma, steps_per_ep):
        self.eps = eps
        self.gamma = gamma
        self.steps_per_ep = steps_per_ep

##Method to initiate agent coordinates should go in setup method since both the Environment and the Agent need to know
#Or we need a hierarchy - consult w/ group

## Q Learn Notes

Rewards come from the environment, agent has values on states that it figures out from the rewards that the environment offers it.

Game has multiple episodes (like, tens of thousands).

Episode has multiple (hundreds?) timesteps. At each episode, we'll have a different location of A from all 24 possible states.

Q table learned across all 24 A locations, 25 agent locations, 4 possible actions, and the pre/post plant situation.

So in total 24 * 25 * 2 states, and four possible actions in each state.

4800 state-action space in total for this toy example.

#### SPECULATIVELY;

If we're constructing this Q-table and using the Bellman equation to backpropagate the rewards from end-state into the Q table, we are effectively "learning" a table which tells us what future rewards are going to be in future states. This lets us, in our _current_ state, choose the action which will best be producing future rewards. So our first near empty Q table will have the agent wander around effectively randomly.


Start with a fixed A and fixed B, to see if you CAN actually learn the Q values. Only THEN do we change to a dynamic location of A.

In [18]:
# Demo

hypers = Hypers(
    eps = 1e-3, 
    gamma = 0.1, 
    steps_per_ep = 100
)

import random as rd
grid = Environment(size = 5)
bond = Agent(grid)
grid.set_agent_start(bond)


for i in range(10):
    new_location = rd.choice(bond.opts)
    grid.move_agent(bond.move(new_location))
    print(grid.grid)
    time.sleep(1)

[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.]]


KeyboardInterrupt: 